In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import words, stopwords

import keras
import pandas as pd
import numpy as np

from keras.preprocessing import text, sequence
import gensim.models.word2vec as w2v
import copy
from tqdm import tqdm
import multiprocessing as mp

import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from string import maketrans 

/home/ubuntu/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
maxlen = 150

In [4]:
train = pd.read_csv("./train_cleaned_no-stopwords.csv")
test = pd.read_csv("./test_cleaned_no-stopwords.csv")

In [5]:
print(train.shape)
print(test.shape)

(159571, 8)
(153164, 2)


In [6]:
list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values

In [7]:
def count_words(corpus):
    word_counts = {}
    for sentence in corpus:
        for word in set(sentence.split()):
                if word not in word_counts: word_counts[word] = 0
                word_counts[word] += 1
    return word_counts

def remove_low_freq_words(text, val_w):
    text = set(text.split())
    text = list(text & val_w)
    if len(text) == 0:
        text = ["CVxTz"]
    text = " ".join(text)    
    return text
        
all_text = list(list_sentences_train) + list(list_sentences_test)
print len(all_text)
word_counts = count_words(all_text)
print len(word_counts)

val_words = set()
for k, v in word_counts.items():
    if v > 1:
        val_words.add(k)
print(len(val_words))
list_sentences_f_train  = ["CVxTz"] * len(list_sentences_train)
list_sentences_f_test   = ["CVxTz"] * len(list_sentences_test)

for i in tqdm(range(len(list_sentences_train))):
    list_sentences_f_train[i] = remove_low_freq_words(list_sentences_train[i], val_words)

for i in tqdm(range(len(list_sentences_test))):
    list_sentences_f_test[i] = remove_low_freq_words(list_sentences_test[i], val_words)

312735


  6%|▌         | 8946/159571 [00:00<00:01, 89404.66it/s]

203883
101897


100%|██████████| 153164/153164 [00:01<00:00, 96132.10it/s]


In [8]:
# Generate common embedings
tokenizer = text.Tokenizer(num_words=len(val_words))
tokenizer.fit_on_texts(list(list_sentences_f_train) + list(list_sentences_f_test))
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_f_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_f_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

list_senteces_all = list_sentences_f_train + list_sentences_f_test
for i in tqdm(range(len(list_senteces_all))):
    list_senteces_all[i] = list_senteces_all[i].split(" ")
    
# hyper parameters of the word2vec model
num_features = 300 # dimensions of each word embedding
min_word_count = 1 # this is not advisable but since we need to extract
# feature vector for each word we need to do this
num_workers = mp.cpu_count() # number of threads running in parallel
context_size = 7 # context window length
downsampling = 1e-3 # downsampling for very frequent words
seed = 1 # seed for random number generator to make results reproducible

word2vec_ = w2v.Word2Vec(
    sg = 1, seed = seed,
    workers = num_workers,
    size = num_features,
    min_count = min_word_count,
    window = context_size,
    sample = downsampling
)

word2vec_.build_vocab(list_senteces_all)
word2vec_.train(list_senteces_all, total_examples = word2vec_.corpus_count, epochs = 10)

embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    if i ==0: print word
    if word in word2vec_.wv:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = word2vec_.wv[word]

Found 98821 unique tokens


In [9]:
list_tokenized_train[0]

[3319,
 12906,
 46,
 8,
 63,
 2376,
 301,
 37,
 228,
 15192,
 652,
 572,
 2782,
 964,
 1122,
 2319,
 4804,
 10037,
 5743,
 39,
 129,
 2,
 6]

In [10]:
list_senteces_all = list_sentences_f_train + list_sentences_f_test
for i in tqdm(range(len(list_senteces_all))):
    list_senteces_all[i] = list_senteces_all[i].split(" ")

100%|██████████| 312735/312735 [00:01<00:00, 194910.22it/s]


In [11]:
list_senteces_all[0]

['retired',
 'vandalisms',
 'since',
 'please',
 'edits',
 'voted',
 'template',
 'new',
 'reverted',
 'dolls',
 'username',
 'explanation',
 'gas',
 'fan',
 'york',
 'fac',
 'hardcore',
 'metallica',
 'closure',
 'made',
 'remove',
 'page',
 'talk']

In [13]:
# hyper parameters of the word2vec model
num_features = 300 # dimensions of each word embedding
min_word_count = 1 # this is not advisable but since we need to extract
# feature vector for each word we need to do this
num_workers = mp.cpu_count() # number of threads running in parallel
context_size = 7 # context window length
downsampling = 1e-3 # downsampling for very frequent words
seed = 1 # seed for random number generator to make results reproducible

In [14]:
word2vec_ = w2v.Word2Vec(
    sg = 1, seed = seed,
    workers = num_workers,
    size = num_features,
    min_count = min_word_count,
    window = context_size,
    sample = downsampling
)

In [15]:
# first we need to built the vocab
word2vec_.build_vocab(list_senteces_all)

In [16]:
word2vec_.train(list_senteces_all, total_examples = word2vec_.corpus_count, epochs = 10)

(78184630, 79928320)

In [17]:
print word2vec_.wv.most_similar('islam')


set(['Zoroastrianism', 'monotheistic', 'shias', 'ilaha', 'apostasy', 'tabari', 'idolatry', "shi'as", 'religious', 'wickedness'])


In [18]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    if i ==0: print word
    if word in word2vec_.wv:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = word2vec_.wv[word]


In [23]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 98822
embed_size = 300
maxlen = 150

inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
model.fit(X_t, y, batch_size=512, epochs=20);

Epoch 1/20
159571/159571 [==============================] - 191s 1ms/step - loss: 0.0427 - acc: 0.9836
Epoch 2/20
159571/159571 [==============================] - 191s 1ms/step - loss: 0.0378 - acc: 0.9850
Epoch 3/20
159571/159571 [==============================] - 191s 1ms/step - loss: 0.0336 - acc: 0.9866
Epoch 4/20
159571/159571 [==============================] - 190s 1ms/step - loss: 0.0303 - acc: 0.9880
Epoch 5/20
159571/159571 [==============================] - 190s 1ms/step - loss: 0.0272 - acc: 0.9893
Epoch 6/20
159571/159571 [==============================] - 190s 1ms/step - loss: 0.0245 - acc: 0.9904
Epoch 7/20
159571/159571 [==============================] - 190s 1ms/step - loss: 0.0220 - acc: 0.9915
Epoch 8/20
159571/159571 [==============================] - 190s 1ms/step - loss: 0.0194 - acc: 0.9925
Epoch 9/20
159571/159571 [==============================] - 190s 1ms/step - loss: 0.0171 - acc: 0.9934
Epoch 10/20
159571/159571 [==============================] - 190s 1ms/ste

In [27]:
X_t_p = model.predict(X_t)

In [28]:
X_te_p = model.predict(X_te)

In [30]:
X_te_p.shape

(153164, 6)

In [29]:
X_t_p.shape

(159571, 6)

In [32]:
pd.DataFrame(np.column_stack([train["id"].values, X_t_p])).to_csv("/home/ubuntu/kaggle/toxicity/team_ensembl/Florian/out/train_1803_FG_LSTMmulticlass.csv", header= False, index= False)
pd.DataFrame(np.column_stack([test["id"].values, X_te_p])).to_csv("/home/ubuntu/kaggle/toxicity/team_ensembl/Florian/out/test_1803_FG_LSTMmulticlass.csv", header= False, index= False)   